In [31]:
!cp out-tinystories/model.eqx out-tinystories/test_model.eqx

In [ ]:
!python train.py config/train_tinystories.py --out_path="out-tinystories/test_model.eqx" --max_iters=250

Overriding config with config/train_tinystories.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_path = 'out-tinystories/model.eqx'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 20
log_interval = 5 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = True # override via command line if you like
tensorboard_log = False # override via command line if you like
log_project = "exp4"
log_run_name = "standard"  # 'run' + str(time.time())

dataset = 'tinystories'
gradient_accumulation_steps = 1
batch_size = 32
block_size = 100 # context of up to 256 previous characters

# baby GPT model :)
n_layer = 16
n_head = 12
n_embd = 512
dropout = 0.0

learning_rate = 1e-4 # with baby networks can afford to go a bit higher
max_iters = 500
lr_decay_iters = 500 # make equal to max_iters usually
min_lr = 1e-5 # learning_

In [5]:
import json
import os
from model import GPT, GPTConfig
import equinox as eqx
import jax
import jax.tree_util as jtu

def load(filename):
    with open(filename, "rb") as f:
        checkpoint_params = json.loads(f.readline().decode())
        gptconf = GPTConfig(**checkpoint_params["model_args"])
        return (
            eqx.tree_deserialise_leaves(
                f, GPT.create_instance(gptconf, key=jax.random.key(1))
            ),
            checkpoint_params,
        )
        
        
model, checkpoint_params = load("out-tinystories/model.eqx")

def model_surgery(path, x):
    if "wte.weight" in jax.tree_util.keystr(path):
        return jax.random.normal(jax.random.key(1), shape=x.shape)
    return x

model = jtu.tree_map_with_path(model_surgery, model)
print(checkpoint_params)
def save(filename, hyperparams, model):
    with open(filename, "wb") as f:
        hyperparam_str = json.dumps(hyperparams)
        f.write((hyperparam_str + "\n").encode())
        eqx.tree_serialise_leaves(f, model)

save("out-tinystories/model2.eqx", checkpoint_params, model)

{'model_args': {'n_layer': 16, 'n_head': 12, 'n_embd': 512, 'block_size': 100, 'bias': False, 'vocab_size': 50304, 'dropout': 0.0}, 'config': {'out_path': 'out-tinystories/model.eqx', 'eval_interval': 250, 'log_interval': 5, 'eval_iters': 20, 'eval_only': False, 'always_save_checkpoint': False, 'init_from': 'scratch', 'wandb_log': False, 'tensorboard_log': True, 'log_project': 'exp1', 'log_run_name': 'gpt2', 'dataset': 'tinystories', 'gradient_accumulation_steps': 1, 'batch_size': 32, 'block_size': 100, 'n_layer': 16, 'n_head': 12, 'n_embd': 512, 'dropout': 0.0, 'bias': False, 'learning_rate': 0.0001, 'max_iters': 250, 'weight_decay': 0.1, 'beta1': 0.9, 'beta2': 0.99, 'grad_clip': 1.0, 'decay_lr': True, 'warmup_iters': 100, 'lr_decay_iters': 500, 'min_lr': 1e-05, 'backend': 'nccl', 'device': 'cuda', 'dtype': 'bfloat16', 'seed': 1}}


In [6]:
!python3 train.py config/train_tinystories.py --init_from="resume" --out_path="out-tinystories/model2.eqx" --max_iters=250

Overriding config with config/train_tinystories.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_path = 'out-tinystories/model.eqx'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 20
log_interval = 5 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like

dataset = 'tinystories'
gradient_accumulation_steps = 1
batch_size = 32
block_size = 100 # context of up to 256 previous characters

# baby GPT model :)
n_layer = 16
n_head = 12
n_embd = 512
dropout = 0.0

learning_rate = 1e-4 # with baby networks can afford to go a bit higher
max_iters = 500
lr_decay_iters = 500 # make equal to max_iters usually
min_lr = 1e-5 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of tokens per iter is small

warmup_iters = 100 # not super necessary po

In [ ]:
import json
import os
from model import GPT, GPTConfig
import equinox as eqx
import jax
import jax.tree_util as jtu

def load(filename):
    with open(filename, "rb") as f:
        checkpoint_params = json.loads(f.readline().decode())
        gptconf = GPTConfig(**checkpoint_params["model_args"])
        return (
            eqx.tree_deserialise_leaves(
                f, GPT.create_instance(gptconf, key=jax.random.key(1))
            ),
            checkpoint_params,
        )
        
        
model, checkpoint_params = load("out-tinystories/model2.eqx")

def model_surgery(path, x):
    if "wte.weight" in jax.tree_util.keystr(path):
        return jax.random.normal(jax.random.key(2), shape=x.shape)
    return x

model = jtu.tree_map_with_path(model_surgery, model)
print(checkpoint_params)
def save(filename, hyperparams, model):
    with open(filename, "wb") as f:
        hyperparam_str = json.dumps(hyperparams)
        f.write((hyperparam_str + "\n").encode())
        eqx.tree_serialise_leaves(f, model)

save("out-tinystories/model3.eqx", checkpoint_params, model)

{'model_args': {'n_layer': 16, 'n_head': 12, 'n_embd': 512, 'block_size': 100, 'bias': False, 'vocab_size': 50304, 'dropout': 0.0}, 'config': {'out_path': 'out-tinystories/model2.eqx', 'eval_interval': 250, 'log_interval': 5, 'eval_iters': 20, 'eval_only': False, 'always_save_checkpoint': False, 'init_from': 'resume', 'wandb_log': False, 'tensorboard_log': True, 'log_project': 'exp1', 'log_run_name': 'gpt2', 'dataset': 'tinystories', 'gradient_accumulation_steps': 1, 'batch_size': 32, 'block_size': 100, 'n_layer': 16, 'n_head': 12, 'n_embd': 512, 'dropout': 0.0, 'bias': False, 'learning_rate': 0.0001, 'max_iters': 250, 'weight_decay': 0.1, 'beta1': 0.9, 'beta2': 0.99, 'grad_clip': 1.0, 'decay_lr': True, 'warmup_iters': 100, 'lr_decay_iters': 500, 'min_lr': 1e-05, 'backend': 'nccl', 'device': 'cuda', 'dtype': 'bfloat16', 'seed': 1}}


In [9]:
!python3 train.py config/train_tinystories.py --init_from="resume" --out_path="out-tinystories/model3.eqx" --max_iters=250

Overriding config with config/train_tinystories.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_path = 'out-tinystories/model.eqx'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 20
log_interval = 5 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like

dataset = 'tinystories'
gradient_accumulation_steps = 1
batch_size = 32
block_size = 100 # context of up to 256 previous characters

# baby GPT model :)
n_layer = 16
n_head = 12
n_embd = 512
dropout = 0.0

learning_rate = 1e-4 # with baby networks can afford to go a bit higher
max_iters = 500
lr_decay_iters = 500 # make equal to max_iters usually
min_lr = 1e-5 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of tokens per iter is small

warmup_iters = 100 # not super necessary po